# Ensemble Learning

## Initial Imports

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter


In [4]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [5]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
## Encode string labels into numbers so model can read and make use of the columns 
le.fit(df["home_ownership"])
df["home_ownership"] = le.transform(df["home_ownership"])

le.fit(df["verification_status"])
df["verification_status"] = le.transform(df["verification_status"])

le.fit(df["pymnt_plan"])
df["pymnt_plan"] = le.transform(df["pymnt_plan"])

le.fit(df["initial_list_status"])
df["initial_list_status"] = le.transform(df["initial_list_status"])

le.fit(df["next_pymnt_d"])
df["next_pymnt_d"] = le.transform(df["next_pymnt_d"])

le.fit(df["application_type"])
df["application_type"] = le.transform(df["application_type"])

le.fit(df["hardship_flag"])
df["hardship_flag"] = le.transform(df["hardship_flag"])

le.fit(df["debt_settlement_flag"])
df["debt_settlement_flag"] = le.transform(df["debt_settlement_flag"])

## Split the Data into Training and Testing

In [7]:
# Create our features
X =  df.drop(columns=['loan_status'])
X = pd.get_dummies(X)

# Create our target
y = df['loan_status']

In [8]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,...,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.0,21.778153,0.217766,0.497697,...,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0,0.371696,0.451066,0.177238
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.0,20.199244,0.718367,0.758122,...,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0,0.483261,0.497603,0.381873
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.0,13.890000,0.000000,0.000000,...,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0,0.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,0.0,19.760000,0.000000,0.000000,...,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0,0.000000,0.000000,0.000000
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,0.0,26.660000,0.000000,1.000000,...,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,0.0,999.000000,18.000000,5.000000,...,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0,1.000000,1.000000,1.000000


In [9]:
# Check the balance of our target values
df.value_counts


<bound method DataFrame.value_counts of        loan_amnt  int_rate  installment  home_ownership  annual_inc  \
0        10500.0    0.1719       375.35               3     66000.0   
1        25000.0    0.2000       929.09               1    105000.0   
2        20000.0    0.2000       529.88               1     56000.0   
3        10000.0    0.1640       353.55               3     92000.0   
4        22000.0    0.1474       520.39               1     52000.0   
...          ...       ...          ...             ...         ...   
68812    10000.0    0.1502       346.76               3     26000.0   
68813    12000.0    0.2727       368.37               3     63000.0   
68814     5000.0    0.1992       185.62               1     52000.0   
68815    40000.0    0.0646      1225.24               1    520000.0   
68816    16000.0    0.1131       350.36               1     72000.0   

       verification_status   issue_d loan_status  pymnt_plan    dti  ...  \
0                        1  Mar

In [10]:
from sklearn.model_selection import train_test_split

# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [11]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


In [12]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [13]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [14]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

random_forest = BalancedRandomForestClassifier(n_estimators = 100, random_state=1)
random_forest = random_forest.fit(X_train, y_train)

In [15]:
# Calculated the balanced accuracy score
y_pred = random_forest.predict(X_test)
balanced_accuracy_score(y_test, y_pred)


0.8078932843427568

In [16]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   62,    25],
       [ 1658, 15460]], dtype=int64)

In [17]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.71      0.90      0.07      0.80      0.63        87
   low_risk       1.00      0.90      0.71      0.95      0.80      0.66     17118

avg / total       0.99      0.90      0.71      0.94      0.80      0.66     17205



In [18]:
# List the features sorted in descending order by feature importance
features_rank = sorted(zip(random_forest.feature_importances_, X.columns), reverse=True)
for feature in features_rank:
    print(f"{feature[1]}: ({feature[0]})")

total_rec_prncp: (0.07919953487669819)
total_rec_int: (0.06231792970789543)
total_pymnt_inv: (0.05920326224010105)
last_pymnt_amnt: (0.05554833485550679)
total_pymnt: (0.05155917183101174)
int_rate: (0.025358263287087048)
out_prncp_inv: (0.019385150067696526)
issue_d_Jan-2019: (0.018046085991905714)
revol_bal: (0.017801346273186393)
dti: (0.017254967803433767)
out_prncp: (0.01706196768390727)
bc_util: (0.016542657008610046)
il_util: (0.015892093862999973)
annual_inc: (0.015783660138203377)
installment: (0.01572324431356952)
tot_hi_cred_lim: (0.015327161537336735)
avg_cur_bal: (0.015108640473288514)
mths_since_recent_inq: (0.01509634613424653)
max_bal_bc: (0.014923741190733659)
issue_d_Mar-2019: (0.014650013226663653)
mo_sin_old_rev_tl_op: (0.014514681639228139)
total_bc_limit: (0.014345926772801948)
bc_open_to_buy: (0.013904776722594386)
total_acc: (0.01320148416482101)
total_il_high_credit_limit: (0.013021733447514885)
open_acc: (0.012657291598040684)
total_rev_hi_lim: (0.012652187894

### Easy Ensemble Classifier

In [19]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
easy = EasyEnsembleClassifier(n_estimators = 100, random_state=1)
easy = easy.fit(X_train, y_train)

In [20]:
# Calculated the balanced accuracy score
y_pred = easy.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9266833460241488

In [21]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   79,     8],
       [  936, 16182]], dtype=int64)

In [22]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.91      0.95      0.14      0.93      0.86        87
   low_risk       1.00      0.95      0.91      0.97      0.93      0.86     17118

avg / total       0.99      0.95      0.91      0.97      0.93      0.86     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.